## Feature extraction

In [1]:
import numpy as np
import pandas as pd

In [2]:
def get_item_numeric_features(df):
    numeric_feature_names = [
        'all_rating_min_max',
        'members_min_max',
        'aired_from_min_max',
        'aired_to_min_max'
    ]
    num_df = df[numeric_feature_names]
    return num_df.to_numpy()

def get_user_numeric_features(df):
    numeric_feature_names = [
        'user_rating_ave_min_max',
        'user_rating_std_min_max',
        'user_aired_from_ave_min_max',
        'user_aired_to_ave_min_max'
    ]
    
    num_df = df[numeric_feature_names]
    return num_df.to_numpy()

In [3]:
def get_multihot_feature(df, feat_name):
    feat_df = df[[feat_name]]
    feat_vecs = feat_df.to_numpy()
    ## This function takes each row v in feat_vecs and extracts the first element, v[0].
    # print(feat_vecs)
    feat_vec = np.apply_along_axis(lambda v: v[0], 1, feat_vecs)
    return feat_vec

In [4]:
def get_label(df):
    label_df = df[['label']]
    return label_df.to_numpy()

In [5]:
def get_all_features(df):
    return ( 
        get_multihot_feature(df, 'genres_multihot'),  ## 43
        get_multihot_feature(df, 'user_liked_genres_multihot'),  ## 43
        get_item_numeric_features(df),   ## 4
        get_user_numeric_features(df)  ## 4
    )

## Load Parguet Dataset

In [6]:
import os

In [7]:
BASE_PATH = '../dataset/'

def data_files():
    filenames = []
    for root, dirs, files in os.walk(BASE_PATH + 'dnn_feat_eng'):
        for file in files:
            if file.endswith('.parquet'):
                print('hhhh')
                filenames.append(os.path.join(root, file))
                
    return filenames

## Define Model
x1: item categorical feature        
x2: user categorical feature          
x3: item numeric features          
x4: user numeric features

In [8]:
# pip install tensorflow


In [9]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorboard.plugins.hparams import api as hp

In [10]:
# HP_LAYERS = hp.HParam("layers", hp.IntInterval(2, 3))
# HP_LAYER_SIZE = hp.HParam("layer_size", hp.Discrete([64, 128, 256]))
# HP_LEARN_RATE = hp.HParam("learn_rate", hp.Discrete([0.001, 0.003, 0.01]))

HP_LAYERS = hp.HParam("layers", hp.IntInterval(2, 3))   ## choose between 2 and 3
HP_LAYER_SIZE = hp.HParam("layer_size", hp.Discrete([64, 128, 256]))
HP_LEARN_RATE = hp.HParam("learn_rate", hp.Discrete([0.001, 0.003, 0.01]))

HPARAMS = [
    HP_LAYERS,
    HP_LAYER_SIZE,
    HP_LEARN_RATE
]             

METRICS = [
    hp.Metric(
        "batch_loss",
        group="train",
        display_name="loss (train)",
    ),
    hp.Metric(
        "loss",
        group="validation",
        display_name="loss (val)",
    ),
]

In [11]:
def build_model(x1_shape, x2_shape, x3_shape, x4_shape, hparams):
    x1_input = keras.layers.Input(shape=(x1_shape,))
    x2_input = keras.layers.Input(shape=(x2_shape,))
    x3_input = keras.layers.Input(shape=(x3_shape,))
    x4_input = keras.layers.Input(shape=(x4_shape,))
    
    # compact embedding for x1 and x2
    compact_x1 = keras.layers.Dense(10)(x1_input)
    compact_x2 = keras.layers.Dense(10)(x2_input)
     
    # concat all
    merge = keras.layers.concatenate([compact_x1, compact_x2, x3_input, x4_input])
    
    # hidden layers
    h_input = merge
    for _ in range(hparams[HP_LAYERS]):
        h = keras.layers.Dense(hparams[HP_LAYER_SIZE], activation='relu')(h_input)
        h_input = h
    
    # output
    output = keras.layers.Dense(1, activation='sigmoid')(h_input)
    
    model = keras.models.Model(inputs=[x1_input, x2_input, x3_input, x4_input], outputs=output)
    
    # optimizer
    opt = keras.optimizers.Adam(learning_rate=hparams[HP_LEARN_RATE])
    model.compile(
        loss='binary_crossentropy',
        optimizer=opt,
        metrics=['accuracy']
    )
    
    return model

## Load Data and Train

In [12]:
test_x1s = []
test_x2s = []
test_x3s = []
test_x4s = []
test_ys = []

In [13]:

def data_files():
    filenames = []
    for root, dirs, files in os.walk(BASE_PATH + 'dnn_feat_eng'):
        for file in files:
            if file.endswith('.parquet'):
                filenames.append(os.path.join(root, file))
                
    return filenames

filenames = data_files()

In [14]:
def run_model(model_id, hparams):
    # build model
    model = build_model(43, 43, 4, 4, hparams)
    print(f"model id: {model_id}:")
    print({h.name: hparams[h] for h in hparams})

    # config hparam logs
    log_filename = f"{model_id}"
    for h in hparams:
        log_filename += f"_{h.name}-{hparams[h]}"
    
    log_dir = os.path.join("hparams", log_filename)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir = log_dir,
        update_freq = 10,
        profile_batch = 0
    )
    hparams_callback = hp.KerasCallback(log_dir, hparams)
    
    # train model
    for filename in filenames[:1]:
        df = pd.read_parquet(filename)

        # shuffle and split train and test
        train_df = df
        train_df = df.sample(frac=0.8, random_state=666)
        test_df = df.drop(train_df.index)

        # get features: multihot, multihot, numeric, numeric
        train_x1, train_x2, train_x3, train_x4 = get_all_features(train_df)
        test_x1, test_x2, test_x3, test_x4 = get_all_features(test_df)

        # get label
        train_y = get_label(train_df)
        test_y = get_label(test_df)

        test_x1s.append(test_x1)
        test_x2s.append(test_x2)
        test_x3s.append(test_x3)
        test_x4s.append(test_x4)
        test_ys.append(test_y)

        print('training on new dataset')

        model.fit(
            [train_x1, train_x2, train_x3, train_x4], 
            train_y, 
            validation_split=0.2,
            batch_size=16, 
            epochs=4,
            callbacks=[tensorboard_callback, hparams_callback]
        )
        return model

        
        

In [15]:

def test_params():
    with tf.summary.create_file_writer('hparams').as_default():
            hp.hparams_config(hparams=HPARAMS, metrics=METRICS)
            
    model_id = 0
    models = []
    for layers in range(HP_LAYERS.domain.min_value, HP_LAYERS.domain.max_value + 1):
        for size in HP_LAYER_SIZE.domain.values:
            for rate in HP_LEARN_RATE.domain.values:
                hparams = {
                    HP_LAYERS: layers,
                    HP_LAYER_SIZE: size,
                    HP_LEARN_RATE: rate
                }
        
                model = run_model(model_id, hparams)
                print(model.layers)         # Lists all layers in the model
                print(model.input_shape)    # Input shape expected by the model
                print(model.output_shape)   # Output shape of the model
                print(model.trainable)      # True if the model is trainable
                print(model.name)           # Name of the model
                model_id += 1
                models.append(model)
    return models


In [16]:
all_models = test_params()
# theModel = all_models[-1]  # Access the last model generated




model id: 0:
{'layers': 2, 'layer_size': 64, 'learn_rate': 0.001}
training on new dataset
Epoch 1/4


/opt/anaconda3/lib/python3.12/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor', 'keras_tensor_1', 'keras_tensor_2', 'keras_tensor_3']. Received: the structure of inputs=('*', '*', '*', '*')
  warnings.warn(


31528/31528 ━━━━━━━━━━━━━━━━━━━━ 18s 541us/step - accuracy: 0.7193 - loss: 0.5470 - val_accuracy: 0.7398 - val_loss: 0.5202
Epoch 2/4
31528/31528 ━━━━━━━━━━━━━━━━━━━━ 17s 547us/step - accuracy: 0.7441 - loss: 0.5155 - val_accuracy: 0.7402 - val_loss: 0.5196
Epoch 3/4
31528/31528 ━━━━━━━━━━━━━━━━━━━━ 17s 542us/step - accuracy: 0.7444 - loss: 0.5147 - val_accuracy: 0.7476 - val_loss: 0.5130
Epoch 4/4
31528/31528 ━━━━━━━━━━━━━━━━━━━━ 17s 538us/step - accuracy: 0.7456 - loss: 0.5124 - val_accuracy: 0.7485 - val_loss: 0.5096
[<InputLayer name=input_layer, built=True>, <InputLayer name=input_layer_1, built=True>, <Dense name=dense, built=True>, <Dense name=dense_1, built=True>, <InputLayer name=input_layer_2, built=True>, <InputLayer name=input_layer_3, built=True>, <Concatenate name=concatenate, built=True>, <Dense name=dense_2, built=True>, <Dense name=dense_3, built=True>, <Dense name=dense_4, built=True>]
[(None, 43), (None, 43), (None, 4), (None, 4)]
(None, 1)
True
functional
model id: 

/opt/anaconda3/lib/python3.12/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_10', 'keras_tensor_11', 'keras_tensor_12', 'keras_tensor_13']. Received: the structure of inputs=('*', '*', '*', '*')
  warnings.warn(


31528/31528 ━━━━━━━━━━━━━━━━━━━━ 18s 547us/step - accuracy: 0.7195 - loss: 0.5481 - val_accuracy: 0.7398 - val_loss: 0.5224
Epoch 2/4
31528/31528 ━━━━━━━━━━━━━━━━━━━━ 17s 545us/step - accuracy: 0.7412 - loss: 0.5195 - val_accuracy: 0.7379 - val_loss: 0.5250
Epoch 3/4
31528/31528 ━━━━━━━━━━━━━━━━━━━━ 17s 542us/step - accuracy: 0.7420 - loss: 0.5182 - val_accuracy: 0.7474 - val_loss: 0.5121
Epoch 4/4
31528/31528 ━━━━━━━━━━━━━━━━━━━━ 154s 5ms/step - accuracy: 0.7435 - loss: 0.5162 - val_accuracy: 0.7449 - val_loss: 0.5147
[<InputLayer name=input_layer_4, built=True>, <InputLayer name=input_layer_5, built=True>, <Dense name=dense_5, built=True>, <Dense name=dense_6, built=True>, <InputLayer name=input_layer_6, built=True>, <InputLayer name=input_layer_7, built=True>, <Concatenate name=concatenate_1, built=True>, <Dense name=dense_7, built=True>, <Dense name=dense_8, built=True>, <Dense name=dense_9, built=True>]
[(None, 43), (None, 43), (None, 4), (None, 4)]
(None, 1)
True
functional_1
mod

...

31528/31528 ━━━━━━━━━━━━━━━━━━━━ 33s 1ms/step - accuracy: 0.7200 - loss: 0.5471 - val_accuracy: 0.7423 - val_loss: 0.5157
Epoch 2/4
31528/31528 ━━━━━━━━━━━━━━━━━━━━ 32s 1ms/step - accuracy: 0.7430 - loss: 0.5171 - val_accuracy: 0.7435 - val_loss: 0.5156
Epoch 3/4
31528/31528 ━━━━━━━━━━━━━━━━━━━━ 35s 1ms/step - accuracy: 0.7449 - loss: 0.5147 - val_accuracy: 0.7460 - val_loss: 0.5125
Epoch 4/4
31528/31528 ━━━━━━━━━━━━━━━━━━━━ 34s 1ms/step - accuracy: 0.7459 - loss: 0.5122 - val_accuracy: 0.7416 - val_loss: 0.5161
[<InputLayer name=input_layer_60, built=True>, <InputLayer name=input_layer_61, built=True>, <Dense name=dense_81, built=True>, <Dense name=dense_82, built=True>, <InputLayer name=input_layer_62, built=True>, <InputLayer name=input_layer_63, built=True>, <Concatenate name=concatenate_15, built=True>, <Dense name=dense_83, built=True>, <Dense name=dense_84, built=True>, <Dense name=dense_85, built=True>, <Dense name=dense_86, built=True>]
[(None, 43), (None, 43), (None, 4), (Non

/opt/anaconda3/lib/python3.12/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_167', 'keras_tensor_168', 'keras_tensor_169', 'keras_tensor_170']. Received: the structure of inputs=('*', '*', '*', '*')
  warnings.warn(


31528/31528 ━━━━━━━━━━━━━━━━━━━━ 33s 1ms/step - accuracy: 0.7190 - loss: 0.5493 - val_accuracy: 0.7431 - val_loss: 0.5183
Epoch 2/4
31528/31528 ━━━━━━━━━━━━━━━━━━━━ 32s 1ms/step - accuracy: 0.7395 - loss: 0.5230 - val_accuracy: 0.7455 - val_loss: 0.5207
Epoch 3/4
31528/31528 ━━━━━━━━━━━━━━━━━━━━ 32s 1ms/step - accuracy: 0.7431 - loss: 0.5191 - val_accuracy: 0.7453 - val_loss: 0.5140
Epoch 4/4
31528/31528 ━━━━━━━━━━━━━━━━━━━━ 32s 1ms/step - accuracy: 0.7442 - loss: 0.5165 - val_accuracy: 0.7333 - val_loss: 0.5279
[<InputLayer name=input_layer_64, built=True>, <InputLayer name=input_layer_65, built=True>, <Dense name=dense_87, built=True>, <Dense name=dense_88, built=True>, <InputLayer name=input_layer_66, built=True>, <InputLayer name=input_layer_67, built=True>, <Concatenate name=concatenate_16, built=True>, <Dense name=dense_89, built=True>, <Dense name=dense_90, built=True>, <Dense name=dense_91, built=True>, <Dense name=dense_92, built=True>]
[(None, 43), (None, 43), (None, 4), (Non

/opt/anaconda3/lib/python3.12/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_178', 'keras_tensor_179', 'keras_tensor_180', 'keras_tensor_181']. Received: the structure of inputs=('*', '*', '*', '*')
  warnings.warn(


31528/31528 ━━━━━━━━━━━━━━━━━━━━ 34s 1ms/step - accuracy: 0.7084 - loss: 0.5642 - val_accuracy: 0.7387 - val_loss: 0.5333
Epoch 2/4
31528/31528 ━━━━━━━━━━━━━━━━━━━━ 33s 1ms/step - accuracy: 0.7364 - loss: 0.5285 - val_accuracy: 0.7383 - val_loss: 0.5256
Epoch 3/4
31528/31528 ━━━━━━━━━━━━━━━━━━━━ 33s 1ms/step - accuracy: 0.7387 - loss: 0.5269 - val_accuracy: 0.7220 - val_loss: 0.5456
Epoch 4/4
31528/31528 ━━━━━━━━━━━━━━━━━━━━ 32s 1ms/step - accuracy: 0.7394 - loss: 0.5261 - val_accuracy: 0.7345 - val_loss: 0.5312
[<InputLayer name=input_layer_68, built=True>, <InputLayer name=input_layer_69, built=True>, <Dense name=dense_93, built=True>, <Dense name=dense_94, built=True>, <InputLayer name=input_layer_70, built=True>, <InputLayer name=input_layer_71, built=True>, <Concatenate name=concatenate_17, built=True>, <Dense name=dense_95, built=True>, <Dense name=dense_96, built=True>, <Dense name=dense_97, built=True>, <Dense name=dense_98, built=True>]
[(None, 43), (None, 43), (None, 4), (Non

In [17]:
# print(len(test_x1s[0]))
# print(len(test_x4s[0]))
# np.array(test_x1s).shape
# np.array(test_x1s[0]).shape

In [18]:
theModel = None
highest_accuracy = 0
for model in all_models:
    test_x1 = np.vstack(test_x1s) if test_x1s else np.array([])
    test_x2 = np.vstack(test_x2s) if test_x2s else np.array([])
    test_x3 = np.vstack(test_x3s) if test_x3s else np.array([])
    test_x4 = np.vstack(test_x4s) if test_x4s else np.array([])
    test_y = np.vstack(test_ys) if test_ys else np.array([])

    # test_x1s[0][0]

    test_loss, test_accuracy = model.evaluate([test_x1, test_x2, test_x3, test_x4], test_y)

    print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))
    if test_accuracy > highest_accuracy:
        highest_accuracy = test_accuracy
        theModel = model
        
print('\nBest Model Test Accuracy:', highest_accuracy)


  502/88672 ━━━━━━━━━━━━━━━━━━━━ 18s 213us/step - accuracy: 0.7541 - loss: 0.4979

/opt/anaconda3/lib/python3.12/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor', 'keras_tensor_1', 'keras_tensor_2', 'keras_tensor_3']. Received: the structure of inputs=('*', '*', '*', '*')
  warnings.warn(


88672/88672 ━━━━━━━━━━━━━━━━━━━━ 17s 192us/step - accuracy: 0.7481 - loss: 0.5084


Test Loss 0.5087596774101257, Test Accuracy 0.747943103313446
  513/88672 ━━━━━━━━━━━━━━━━━━━━ 18s 205us/step - accuracy: 0.7549 - loss: 0.5046

/opt/anaconda3/lib/python3.12/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_10', 'keras_tensor_11', 'keras_tensor_12', 'keras_tensor_13']. Received: the structure of inputs=('*', '*', '*', '*')
  warnings.warn(


88672/88672 ━━━━━━━━━━━━━━━━━━━━ 17s 191us/step - accuracy: 0.7463 - loss: 0.5132


Test Loss 0.5135372281074524, Test Accuracy 0.7460399866104126
  787/88672 ━━━━━━━━━━━━━━━━━━━━ 17s 199us/step - accuracy: 0.7462 - loss: 0.5114

...

88672/88672 ━━━━━━━━━━━━━━━━━━━━ 21s 229us/step - accuracy: 0.7270 - loss: 0.5422


Test Loss 0.5426263809204102, Test Accuracy 0.7267237305641174
  283/88672 ━━━━━━━━━━━━━━━━━━━━ 33s 382us/step - accuracy: 0.7489 - loss: 0.4982

/opt/anaconda3/lib/python3.12/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_156', 'keras_tensor_157', 'keras_tensor_158', 'keras_tensor_159']. Received: the structure of inputs=('*', '*', '*', '*')
  warnings.warn(


88672/88672 ━━━━━━━━━━━━━━━━━━━━ 29s 328us/step - accuracy: 0.7426 - loss: 0.5149


Test Loss 0.5152139663696289, Test Accuracy 0.7423861026763916
  291/88672 ━━━━━━━━━━━━━━━━━━━━ 32s 363us/step - accuracy: 0.7476 - loss: 0.5084

/opt/anaconda3/lib/python3.12/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_167', 'keras_tensor_168', 'keras_tensor_169', 'keras_tensor_170']. Received: the structure of inputs=('*', '*', '*', '*')
  warnings.warn(


88672/88672 ━━━━━━━━━━━━━━━━━━━━ 29s 332us/step - accuracy: 0.7331 - loss: 0.5269


Test Loss 0.5272397398948669, Test Accuracy 0.7328643202781677
88672/88672 ━━━━━━━━━━━━━━━━━━━━ 29s 329us/step - accuracy: 0.7342 - loss: 0.5297


Test Loss 0.5299948453903198, Test Accuracy 0.7339110374450684

Best Model Test Accuracy: 0.7492942810058594


## Test


In [20]:
theModel.save('mlp_model.keras')

In [21]:

n = 1
g = 0

for i in range(n):
    expect = test_y[i][0]
    x1 = test_x1[i]
    x2 = test_x2[i]
    x3 = test_x3[i]
    x4 = test_x4[i]
    xs = [ np.array([x1]), np.array([x2]), np.array([x3]), np.array([x4]) ]
    print('xs')
    print(xs)
    
    predict = theModel.predict(xs)
    predict = predict[0][0]
    if predict > 0.5:
        predict = 1
    else:
        predict = 0
        
    if predict == expect:
        g += 1.0
    
#     print(f'Expect {expect}, predict {predict}')
    
    
print()
print('accuracy:')
print(g / n)

xs
[array([[0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int32), array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int32), array([[0.87, 0.03, 0.61, 0.61]], dtype=float32), array([[0., 0., 0., 0.]], dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step

accuracy:
1.0


In [67]:
import numpy as np

array1 = np.array([[1, 2, 3],
                   [4, 5, 6]])

array2 = np.array([[7, 8, 9],
                   [10, 11, 12]])

array3 = np.array([[13, 14, 15],
                   [16, 17, 18]])

stacked_array = np.vstack([array1, array2, array3])
print(stacked_array)
stacked_array.shape

[[ 1  2  3]
 [ 4  5  6]
 [ 7  8  9]
 [10 11 12]
 [13 14 15]
 [16 17 18]]


(6, 3)